In [ ]:
%%capture
!pip install pycaret[full]

In [ ]:
## The magic four
import pandas as pd
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt 

#garbage collection (clear up some RAM)
import gc

#pycaret
from pycaret.classification import *

#cuML
import cudf

#TPOT
from tpot import TPOTClassifier

%matplotlib inline

In [ ]:
#this is an aesthetic choice and just removes the many warnings that some functions and comands produce
#it helps significantly declutter the workbook
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#train_df = pd.read_csv('../input/tabular-playground-series-oct-2021/train.csv', index_col='id')
#test_df = pd.read_csv('../input/tabular-playground-series-oct-2021/test.csv', index_col='id')
#cudf_train = cudf.DataFrame.from_pandas(train)

In [ ]:
train = cudf.read_csv('../input/tabular-playground-series-oct-2021/train.csv', index_col='id')
test = cudf.read_csv('../input/tabular-playground-series-oct-2021/test.csv', index_col='id')

gc.collect()

In [ ]:
#saving the index of the test dataset for later use
#idx = test.index

In [ ]:
try:
    floats = ['f' + str(i) for i in range(242)]
    ints = ['f' + str(i) for i in range(242, 285)]
    for i in train.columns:
        if i in floats:
            train[i] = train[i].astype(np.float32)
        elif i in ints:
            train[i] = train[i].astype(np.int8)
    for j in test.columns:
        if j in floats:
            test[j] = test[j].astype(np.float32)
        elif j in ints:
            test[j] = test[j].astype(np.int8)
except:
    print('fail')
finally:
    del floats, ints
    gc.collect()

In [ ]:
#setting up the pipeline
clf = setup(data = train.to_pandas().sample(frac = 0.1), 
            target = 'target',
            data_split_stratify = True,
            use_gpu = 'force',
            fold = 5,
            n_jobs = -1, 
            silent = True
           )
gc.collect()

In [ ]:
#compare_models(exclude = ['dt','ada','gbc','et','svm','lda','qda'], sort = 'AUC')

In [ ]:
compare_models(sort = 'AUC')

In [ ]:
gc.collect()

In [ ]:
catboost = create_model('catboost')

In [ ]:
catboost = tune_model(catboost, optimize = 'auc', choose_better = True, search_library = 'optuna')

In [ ]:
predict_model(catboost);

In [ ]:
final_catboost = finalize_model(catboost)

In [ ]:
predict_model(final_catboost);

In [ ]:
plot_model(final_catboost, plot = 'auc')

In [ ]:
plot_model(final_catboost, plot = 'pr')

In [ ]:
plot_model(final_catboost, plot = 'feature')

In [ ]:
plot_model(final_catboost, plot = 'confusion_matrix')

In [ ]:
#prediction on unseen data
unseen_predictions_cat = predict_model(final_catboost, data=test.to_pandas(), raw_score = True)
unseen_predictions_cat.head()

In [ ]:
#top3 = compare_models(include = ['catboost', 'xgboost','lightgbm'], sort = 'AUC', n_select = 3)
#gc.collect()

In [ ]:
#top3v2 = compare_models(sort = 'AUC', n_select = 3)
#gc.collect()

In [ ]:
#blend = blend_models(top3)
#gc.collect()

In [ ]:
#predict_model(blend);
#gc.collect()

In [ ]:
#final_blend = finalize_model(blend)
#gc.collect()

In [ ]:
#predict_model(final_blend);
#gc.collect()

In [ ]:
#plot_model(final_blend, plot = 'auc')

In [ ]:
#plot_model(final_blend, plot = 'pr')

In [ ]:
#plot_model(final_blend, plot = 'feature')

In [ ]:
#plot_model(final_blend, plot = 'confusion_matrix')

In [ ]:
#prediction on unseen data
unseen_predictions_blend = predict_model(final_blend, data=test, raw_score = True)
unseen_predictions_blend.head()
gc.collect()

In [ ]:
tpot = TPOTClassifier(
    generations=1,
    config_dict="TPOT cuML",
    n_jobs=1, # cuML requires n_jobs=1, the default
    scoring = 'roc_auc',
    verbosity=2)

In [ ]:
X = train.to_pandas().sample(frac = 0.005)
y = X.pop('target')
gc.collect()

In [ ]:
tpot.fit(X, y)

In [ ]:
tpot.score(X,y)

In [ ]:
preds = tpot.predict(test.to_pandas())

In [ ]:
assert(len(test.index)==len(unseen_predictions_cat))

sub = pd.DataFrame(list(zip(test.to_pandas().index, unseen_predictions_cat.Score_1)),columns = ['id', 'target'])

sub.to_csv('submission.csv', index = False)

print(sub)